# BIDS conversion

**Author:** Thuy Dao

**Date:** 23 Nov 2024

### Citation and Resources:
#### Tools included in this notebook

__dcm2niix:__
- Li, X., Morgan, P. S., Ashburner, J., Smith, J., & Rorden, C. (2016). The first step for neuroimaging data analysis: DICOM to NIfTI conversion. Journal of Neuroscience Methods, 264, 47–56. [https://doi.org/10.1016/j.jneumeth.2016.03.001](https://doi.org/10.1016/j.jneumeth.2016.03.001)

  
__BIDScoin:__
- Zwiers MP, Moia S and Oostenveld R (2022) BIDScoin: A User-Friendly Application to Convert Source Data to Brain Imaging Data Structure. Front. Neuroinform. 15:770608. doi: [10.3389/fninf.2021.770608](https://doi.org/10.3389/fninf.2021.770608)

__HeuDiConv:__
- Yaroslav O. Halchenko, Mathias Goncalves, Satrajit Ghosh, Pablo Velasco, Matteo Visconti di Oleggio Castello, Taylor Salo, John T. Wodder II, Michael Hanke, Patrick Sadil, Krzysztof Jacek Gorgolewski, Horea-Ioan Ioanas, Chris Rorden, Timothy J. Hendrickson, Michael Dayan, Sean Dae Houlihan, James Kent, Ted Strauss, John Lee, Isaac To, … David N. Kennedy. (2024). HeuDiConv — flexible DICOM conversion into structured directory layouts (v1.3.1). Zenodo. [https://doi.org/10.5281/zenodo.13994097](https://doi.org/10.5281/zenodo.13994097)

#### Dataset
This example uses a phantom dataset generated by the University of Arizona on their Siemens Skyra 3T with Syngo MR VE11c software on 2018_02_08.

## BIDS conversion
1. In this interactive example notebook, we first download DICOM data using `datalad`.

2. Next we will convert DICOMS into the BIDS data structure using:
    - 2.1. `dcm2niix` which converts data to NIfTi and then manually organizes the folder.
    - 2.2. `heudiconv` to automate the conversion and organization with a provided heuristic file.
    - 2.3. `bidscoin` to automate the conversion and organization with the bidseditor interface (optionally using `--automated` to disable user interaction).

## 1. Data preparation

Download DICOM data from Datalad.

In [1]:
%%bash
datalad install https://datasets.datalad.org/repronim/heudiconv-reproin-example
cd heudiconv-reproin-example/REPROIN/dicom && datalad get 001

[INFO] Attempting a clone into /home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv-reproin-example 
[INFO] Attempting to clone from https://datasets.datalad.org/repronim/heudiconv-reproin-example to /home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv-reproin-example 
[INFO] Attempting to clone from https://datasets.datalad.org/repronim/heudiconv-reproin-example/.git to /home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv-reproin-example 
[INFO] Start enumerating objects 
[INFO] Start counting objects 
[INFO] Start compressing objects 
[INFO] Start receiving objects 
[INFO] Start resolving deltas 
[INFO] Completed clone attempts for Dataset(/home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv-reproin-example) 
[INFO] scanning for unlocked files (this may take some time) 


install(ok): /home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv-reproin-example (dataset)
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0001-0001.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0001-0002.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0001-0003.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0001-0004.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0001-0005.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0002-0001.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0002-0002.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Coben - 1/Localizers_4/IM-0012-0002-0003.dcm (file) [from origin...]
get(ok): REPROIN/dicom/001/Patterson_Co

We will be working from a directory called `REPROIN`. Under the REPROIN directory is the `dicom` subdirectory: Under the subject number `001` the session `Patterson_Coben - 1` is nested. Each dicom sequence folder is nested under the session:

In [2]:
!tree heudiconv-reproin-example/REPROIN/ -I "*.dcm"

heudiconv-reproin-example/REPROIN/
└── dicom
    └── 001
        └── Patterson_Coben - 1
            ├── Localizers_4
            ├── anatT1w_acqMPRAGE_6
            ├── dwi_dirAP_9
            ├── fmap_acq4mm_7
            ├── fmap_acq4mm_8
            ├── fmap_dirPA_15
            └── func_taskrest_16

10 directories, 0 files


## 2. Convert DICOMS to BIDS-formatted NIfTI

### 2.1. Using dcm2niix (manual workflow)

`dcm2niix` converts neuroimaging data from the DICOM format to the NIfTI format and generates a BIDS JSON format sidecar which includes relevant information. However, users need to manually organize these files into a bids-compliant subject directory.

#### 2.1.1. Convert DICOM to NIfTI

In [3]:
import module
await module.load('dcm2niix/v1.0.20240202')

In [4]:
!dcm2niix -h

Chris Rorden's dcm2niiX version v1.0.20240202  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.4.0 x86-64 (64-bit Linux)
usage: dcm2niix [options] <in_folder>
 Options :
  -1..-9 : gz compression level (1=fastest..9=smallest, default 6)
  -a : adjacent DICOMs (images from same series always in same folder) for faster conversion (n/y, default n)
  -b : BIDS sidecar (y/n/o [o=only: no NIfTI], default y)
   -ba : anonymize BIDS (y/n, default y)
  -c : comment stored in NIfTI aux_file (up to 24 characters e.g. '-c VIP', empty to anonymize e.g. 0020,4000 e.g. '-c ""')
  -d : directory search depth. Convert DICOMs in sub-folders of in_folder? (0..9, default 5)
  -e : export as NRRD (y) or MGH (o) or JSON/JNIfTI (j) or BJNIfTI (b) instead of NIfTI (y/n/o/j/b, default n)
  -f : filename (%a=antenna (coil) name, %b=basename, %c=comments, %d=description, %e=echo number, %f=folder name, %g=accession number, %i=ID of patient, %j=seriesInstanceUID, %k=studyInstanceUID, %m=manufacturer, %n=name of patient, %o=me

In [5]:
!mkdir dcm2niix_bids
!dcm2niix n -o ./dcm2niix_bids ./heudiconv-reproin-example/REPROIN/dicom

Chris Rorden's dcm2niiX version v1.0.20240202  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.4.0 x86-64 (64-bit Linux)
Found 791 DICOM file(s)
Slices not stacked: orientation varies (vNav or localizer?) [0.0964069 0.780133 0.618141 0.0553476 0.615871 -0.7859] != [-0.522168 0.736033 0.430808 0.0695744 0.540223 -0.838641]
Convert 2 DICOM as ./dcm2niix_bids/dicom_anat-scout_20190208142838_3_i00001 (162x162x2x1)
Convert 1 DICOM as ./dcm2niix_bids/dicom_anat-scout_20190208142838_3_i00004 (162x162x1x1)
Convert 1 DICOM as ./dcm2niix_bids/dicom_anat-scout_20190208142838_3_i00003 (162x162x1x1)
Convert 1 DICOM as ./dcm2niix_bids/dicom_anat-scout_20190208142838_3_i00002 (162x162x1x1)
Convert 128 DICOM as ./dcm2niix_bids/dicom_anat-scout_20190208142838_1 (160x160x128x1)
Convert 128 DICOM as ./dcm2niix_bids/dicom_anat-scout_20190208142838_2 (160x160x128x1)
Convert 4 DICOM as ./dcm2niix_bids/dicom_func_task-rest_20190208142838_16 (64x64x44x4)
Convert 132 DICOM as ./dcm2niix_bids/dicom_fmap_dir-PA_2019020814283

#### 2.1.2. Organize data into BIDS structure

In [6]:
%%bash
mkdir -p ./dcm2niix_bids/anat
mkdir -p ./dcm2niix_bids/fmap
mkdir -p ./dcm2niix_bids/dwi
mkdir -p ./dcm2niix_bids/func

find ./dcm2niix_bids -maxdepth 1 -type f -name *anat* -exec mv {} ./dcm2niix_bids/anat \;
find ./dcm2niix_bids -maxdepth 1 -type f -name *fmap* -exec mv {} ./dcm2niix_bids/fmap \;
find ./dcm2niix_bids -maxdepth 1 -type f -name *dwi* -exec mv {} ./dcm2niix_bids/dwi \;
find ./dcm2niix_bids -maxdepth 1 -type f -name *func* -exec mv {} ./dcm2niix_bids/func \;

In [7]:
!tree ./dcm2niix_bids/

./dcm2niix_bids/
├── anat
│   ├── dicom_anat-T1w_acq-MPRAGE_20190208142838_6.json
│   ├── dicom_anat-T1w_acq-MPRAGE_20190208142838_6.nii
│   ├── dicom_anat-scout_20190208142838_1.json
│   ├── dicom_anat-scout_20190208142838_1.nii
│   ├── dicom_anat-scout_20190208142838_2.json
│   ├── dicom_anat-scout_20190208142838_2.nii
│   ├── dicom_anat-scout_20190208142838_3_i00001.json
│   ├── dicom_anat-scout_20190208142838_3_i00001.nii
│   ├── dicom_anat-scout_20190208142838_3_i00002.json
│   ├── dicom_anat-scout_20190208142838_3_i00002.nii
│   ├── dicom_anat-scout_20190208142838_3_i00003.json
│   ├── dicom_anat-scout_20190208142838_3_i00003.nii
│   ├── dicom_anat-scout_20190208142838_3_i00004.json
│   ├── dicom_anat-scout_20190208142838_3_i00004.nii
│   ├── dicom_anat-scout_20190208142838_4_i00001.json
│   ├── dicom_anat-scout_20190208142838_4_i00001.nii
│   ├── dicom_anat-scout_20190208142838_4_i00002.json
│   ├── dicom_anat-scout_20190208142838_4_i00002.nii
│   ├── dicom_anat-scout_2019020814

### 2.2. Using HeuDiConv (automated workflow)
Typical use of heudiconv will require the creation and editing of your heuristics file.

But heudiconv also comes with existing heuristics which can be used as is, like `convertall` or `ReproIn` could be immediately reused and represent two ends of the spectrum in heuristics:
- `convertall` is very simple and does not automate anything – it is for a user to modify filenames in the prepared conversion table, and then rerun with `-c dcm2niix`
- `reproin` can be used fully automated, if original sequences were named according to its ReproIn convention.

In [8]:
await module.load('heudiconv/1.3.1')

In [9]:
!heudiconv -f reproin -b -o ./heudiconv_reproin_bids --files ./heudiconv-reproin-example/REPROIN/dicom/001

INFO: Running heudiconv version 1.3.1 latest Unknown
INFO: Analyzing 791 dicoms
INFO: Filtering out 0 dicoms based on their filename
INFO: Generated sequence info for 1 studies with 11 entries total
INFO: Processing sequence infos to deduce study/session
INFO: Study session for StudySessionInfo(locator='Patterson/Coben', session=None, subject='001')
INFO: Need to process 1 study sessions
INFO: PROCESSING STARTS: {'subject': '001', 'outdir': '/home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv_reproin_bids/Patterson/Coben', 'session': None}
INFO: Processing 11 pre-sorted seqinfo entries
INFO: Processing 11 seqinfo entries
INFO: Doing conversion using dcm2niix
INFO: Converting /home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv_reproin_bids/Patterson/Coben/sub-001/anat/sub-001_scout (3 DICOMs) -> /home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv_reproin_bids/Patterson/Coben/sub-001/anat . Converter: dcm2niix . Output typ

### 2.3. Using bidscoin (automated workflow)

BIDScoin requires the source data repository to be organized according to a `subject/[session]/data` structure where `data` folder can be structured in DICOM series layout, DICOMDIR layout, flat DICOM layout. Refer to [this doc](https://bidscoin.readthedocs.io/en/stable/preparation.html) for more details.

To perform BIDS conversion, run:

- (1a) `bidsmapper` command
- once `bidsmapper` finishes, it automatically launches (1b) the `bidseditor`. *Note*: The `--automated` flag skips the manual editing step with bidseditor
- (2) `bidscoiner` command

![image.png](https://bidscoin.readthedocs.io/en/stable/_images/bidsmap_flow.png)


In [10]:
import module
await module.load('bidscoin/4.3.3')

In [11]:
%%bash
mkdir bidscoin_bids
bidsmapper ./heudiconv-reproin-example/REPROIN/dicom ./bidscoin_bids --subprefix '00' --sesprefix 'Patterson_Coben' --automated  # Scans your data and creates a study bidsmap non-interactively
bidscoiner ./heudiconv-reproin-example/REPROIN/dicom ./bidscoin_bids    # Converts your data to BIDS using the study bidsmap

INFO | 
INFO | -------------- START BIDSmapper ------------
INFO | >>> bidsmapper sourcefolder=/home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv-reproin-example/REPROIN/dicom bidsfolder=/home/jovyan/Git_repositories/example-notebooks/books/workflows/bidscoin_bids bidsmap=bidsmap.yaml template=/home/jovyan/.bidscoin/4.3.3+qt5/templates/bidsmap_dccn.yaml plugins=[] subprefix=00 sesprefix=Patterson_Coben store=False force=False
INFO | No existing bidsmap file found: /home/jovyan/Git_repositories/example-notebooks/books/workflows/bidscoin_bids/code/bidscoin/bidsmap.yaml
INFO | Reading: /home/jovyan/.bidscoin/4.3.3+qt5/templates/bidsmap_dccn.yaml
INFO | Checking the bidsmap run-items:
SUCCESS | All datatypes and options in the template bidsmap are valid
INFO | Mapping: /home/jovyan/Git_repositories/example-notebooks/books/workflows/heudiconv-reproin-example/REPROIN/dicom/001/Patterson_Coben - 1 (subject 1/1)
VERBOSE | Executing plugin: dcm2niix2bids -> /home/jovyan/G

In [12]:
!tree ./bidscoin_bids/

./bidscoin_bids/
├── README
├── code
│   └── bidscoin
│       ├── bidscoiner.errors
│       ├── bidscoiner.log
│       ├── bidscoiner.tsv
│       ├── bidsmap.yaml
│       ├── bidsmapper.errors
│       └── bidsmapper.log
├── dataset_description.json
├── participants.json
├── participants.tsv
└── sub-1
    └── ses-1
        ├── anat
        │   ├── sub-1_ses-1_acq-anatT1wacqMPRAGE_T1w.json
        │   └── sub-1_ses-1_acq-anatT1wacqMPRAGE_T1w.nii.gz
        ├── dwi
        │   ├── sub-1_ses-1_acq-dwidirAP_dir-AP_dwi.bval
        │   ├── sub-1_ses-1_acq-dwidirAP_dir-AP_dwi.bvec
        │   ├── sub-1_ses-1_acq-dwidirAP_dir-AP_dwi.json
        │   └── sub-1_ses-1_acq-dwidirAP_dir-AP_dwi.nii.gz
        ├── fmap
        │   ├── sub-1_ses-1_acq-fmapacq4mm_magnitude1.json
        │   ├── sub-1_ses-1_acq-fmapacq4mm_magnitude1.nii.gz
        │   ├── sub-1_ses-1_acq-fmapacq4mm_magnitude2.json
        │   ├── sub-1_ses-1_acq-fmapacq4mm_magnitude2.nii.gz
        │   ├── sub-1_ses-1_acq-fmapacq4mm_pha

### Dependencies in Jupyter/Python
- Using the package [watermark](https://github.com/rasbt/watermark) to document system environment and software versions used in this notebook

In [13]:
%load_ext watermark

%watermark
%watermark --iversions

Last updated: 2025-10-31T03:32:37.738762+00:00

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.16.1

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.4.0-204-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 32
Architecture: 64bit


